# CZ gate  

In [1]:
from system import *
s1 = system('o-x-o',MMA=True)

Initializing system o-x-o  ...
Constructing states and excitations...
Constructing ground and first-excited statespace...
Constructing gs_hamiltonian ...
Constructing e1_hamiltonian ...
Constructing interactions V_plus and V_minus ...
Constructing NJ_hamiltonian  ...
Inverting NJ_hamiltonian  ...
Constructing eff_hamiltonian and effective lindblau operators ...

System  o-x-o  initialized in 18.8 seconds.


# $\Delta_{E\gamma} = \ \dfrac{\Delta_E}{\gamma}\, C^n \qquad \Delta_{e\gamma} = \ \dfrac{\Delta_e}{\gamma}\, C^m$

In [10]:
n = 1/2
m = 1/2

# Effective Hamiltonian Matrix obtain in $C \longrightarrow \infty$ limit

In [11]:
var('gamma','DE','De','g','g_f','Omega','v','gamma_f','gamma_g',domain='positive')
var('DEt',domain='complex',  latex_name =r'\tilde{\Delta }_E')
var('Det',domain='complex',  latex_name =r'\tilde{\Delta }_e')
var('DEg',domain='positive',  latex_name =r'\Delta_{E\gamma}')
var('Deg',domain='positive',  latex_name =r'\Delta_{e\gamma}')
var('kappa_c','kappa_b',domain='positive')
var('C',domain='positive')
var('c_k',domain='positive',  latex_name =r'c_\kappa')

eff_hamiltonian_tayl = sg.copy( s1.eff_hamiltonian.parent().zero())
for diag in s1.pos_gs:
    print(f'Calculating {diag} out of {s1.pos_gs}...')
    #Take only one of the compl conj terms. Switch maybe second 0 to 1
    min_expr_len = 10**10
    for term in range(1):
        b = s1.eff_hamiltonian[diag,diag]

        b = b.operands()[1].operands()[term]
        #b = b.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
        #b = b.subs(De=+0.5*I*gamma   +Det*gamma)
        b = b.subs(DE=DEg*gamma*C^n)
        b = b.subs(De=Deg*gamma*C^m)
        
        var('R_f',domain='positive')#ratio  (g_f/g)^2
        b = b.subs(g_f=g*sqrt(R_f))

        var('R_v',domain='positive',  latex_name =r'R_{\nu}') #ratio (v/g)^2
        b = b.subs(v=g*sqrt(R_v))

        var('r_b',domain='positive')
        b = b.subs(kappa_b=gamma/r_b)
        
        var('r_f',domain='positive',latex_name =r'r_f')
        b = b.subs(gamma_f=gamma*r_f)
        
        var('r_g',domain='positive',latex_name =r'r_g')
        b = b.subs(gamma_g=gamma*r_g)
        
        b = b.subs(g = sqrt(C*gamma*kappa_c)) 
        b = b.subs(r_b = c_k/C)

        b = b*gamma  #Omega has to be substituted with Omega/gamma


        b = b._mathematica_().Factor()._sage_()
        b = MMA_simplify(b)#,full=True)
        #holds the smallest expression
        if len(str(b))<min_expr_len:
            min_expr_len = len(str(b))
            h_eff = b
  
    q = h_eff.subs(C=1/x)
    q = MMA_simplify(q)
    tayl = q._mathematica_().Series((x,0,1)).Normal()._sage_()
    tayl = tayl.subs(x=1/C)
    
    #tayl =  (tayl + conjugate(tayl))/2      # try with + conjugate_transp

    #taylr = taylr.expand()
    #tayl = MMA_simplify(tayl,full=True)
    eff_hamiltonian_tayl[diag,diag] = Omega^2/gamma * tayl

print('\nReal part of H_eff in ground state subspace taylor expanded in 1/C: ')
show(symround(eff_hamiltonian_tayl[s1.pos_gs,s1.pos_gs],digits=4))

eff_hamiltonian_tayl = (eff_hamiltonian_tayl[s1.pos_gs,s1.pos_gs] + conjugate(eff_hamiltonian_tayl[s1.pos_gs,s1.pos_gs]))/2

Calculating 0 out of [0, 1, 16, 17]...
Calculating 1 out of [0, 1, 16, 17]...
Calculating 16 out of [0, 1, 16, 17]...
Calculating 17 out of [0, 1, 16, 17]...

Real part of H_eff in ground state subspace taylor expanded in 1/C: 


[0.7500*(-2*I*R_v^2*c_k^2 - 0.6667*I*R_v*c_k - 0.04167*I)*Omega^2/((R_v^2*c_k^2 + 0.5000*R_v*c_k + 0.06250)*C*R_f*gamma)                                                                                                                       0                                                                                                                       0                                                                                                                       0]
[                                                                                                                      0                                  -2*I*(R_v^2*c_k^2 + 0.7500*R_v*c_k + 0.06250)*Omega^2/((R_v*c_k + 0.2500)*C*R_f*gamma)                                                                                                                       0                                                                                                                       0]
[                                                                                                                      0                                                                                                                       0                                  -2*I*(R_v^2*c_k^2 + 0.7500*R_v*c_k + 0.06250)*Omega^2/((R_v*c_k + 0.2500)*C*R_f*gamma)                                                                                                                       0]
[                                                                                                                      0                                                                                                                       0                                                                                                                       0                                                                            4*I*(R_v*c_k + 0.1250)*Omega^2/(C*R_f*gamma)]

# CZ gate time

In [12]:
#l_0-l_1-l_2+l_3
denom = eff_hamiltonian_tayl[0,0]+eff_hamiltonian_tayl[3,3]-eff_hamiltonian_tayl[1,1]-eff_hamiltonian_tayl[2,2]

t_g = np.pi/denom
t_g = MMA_simplify(t_g,full=False) #not full if too slow
t_g = SR(str(t_g).replace('Sqrt','sqrt')).simplify()

TypeError: Error executing code in Mathematica
CODE:
	sage5=Simplify[sage2];
Mathematica ERROR:
	Simplify::infd: Expression <<1>> simplified to ComplexInfinity.

In [ ]:
show(symround((t_g)))
#GHZ REGIME 
rf =1 
de=122183415123789/100000000000000
r_v=1.01014311134038/c_k
#show(symround((t_g.subs(R_f =rf ,DEg=de,R_v=r_v))))

# Lindblau Operators in $C \longrightarrow \infty$ limit

Choose which of the Lindblau operators to view and which non zero element of it

In [6]:
lind_op_number = len(s1.lindblau_list)

eff_lind_taylor = []
eff_lind = []
eff_lind_coeff = []
for lind_op in range(lind_op_number):
    print(f'{lind_op} out of {lind_op_number-1}')
    eff_lind_taylor.append([])
    eff_lind.append([])
    L_matrix = s1.eff_lindblau_list[lind_op]
    L_nonzeros = []
    L_nonzeros_pos = []
    for i in  range(L_matrix.nrows()):
        for j in  range(L_matrix.ncols()):
            if not L_matrix[i,j].is_zero():
                L_nonzeros.append(L_matrix[i,j])
                L_nonzeros_pos.append((i,j))
    eff_lind_coeff.append(s1.L_coeffs[lind_op])
    for which in range(len(L_nonzeros)):

        L_elem = L_nonzeros[which]

        #L_elem = L_elem.subs(DE=+0.5*I*gamma_f + 0.5*I*gamma_g + DEt*gamma)
        #L_elem = L_elem.subs(De=+0.5*I*gamma   +Det*gamma)
        L_elem = L_elem.subs(DE=DEg*gamma*C^n)
        L_elem = L_elem.subs(De=Deg*gamma*C^m)
        #COUPLINGS

        L_elem = L_elem.subs(g_f=g*sqrt(R_f))

        L_elem = L_elem.subs(v=g*sqrt(R_v))

        #ABSORBTIONS
        L_elem = L_elem.subs(kappa_b=gamma/r_b)

        L_elem = L_elem.subs(gamma_g=gamma*r_g)

        L_elem = L_elem.subs(gamma_f=gamma*r_f)

        L_elem = L_elem.subs(g = sqrt(C*gamma*kappa_c)) 
        L_elem = L_elem.subs(r_b = c_k/C)
        
        L_elem = L_elem._mathematica_().Factor()._sage_()


        L_elem = MMA_simplify(L_elem)#,full=True)

        L_elem = SR(str(L_elem).replace('Sqrt','sqrt'))
        
        L_elem_show = symround(L_elem)
        
        eff_lind[lind_op].append(L_elem_show)
        
        var('x')
        q = L_elem.subs(C=1/x)
        q = MMA_simplify(q)
        tayl = q._mathematica_().Series((x,0,1)).Normal()._sage_()
        tayl = tayl.subs(x=1/C)
        tayl = SR(str(tayl).replace('Sqrt','sqrt'))
        tayl_show = symround(tayl)
        
        eff_lind_taylor[-1].append(tayl_show)

0 out of 8
1 out of 8
symround: Deleted coefficient 1.1e-14
symround: Deleted coefficient 1.1e-14
symround: Deleted coefficient 1.3e-15
2 out of 8
3 out of 8
4 out of 8
5 out of 8
6 out of 8
7 out of 8
8 out of 8
symround: Deleted coefficient 1.1e-14
symround: Deleted coefficient 1.1e-14
symround: Deleted coefficient 1.3e-15


# See results

In [7]:
#show((eff_lind_taylor[2][3]))

In [8]:
#show(eff_lind[0][3])

In [9]:
lindop = 4
a = eff_lind_taylor[lindop][0]
err = t_g*a*conjugate(a)
err = SR(str(MMA_simplify( err,full=False )).replace('Sqrt','sqrt')).simplify()
show( symround( err ) )
print(f'coeff:')
show(eff_lind_coeff[lindop])

-2.0*((0.66*I*R_v^4*c_k^4 + 0.55*I*R_v^3*c_k^3 + 0.17*I*R_v^2*c_k^2 + 0.021*I*R_v*c_k + 0.00086*I)*C^1.5*R_f + (R_v^4*c_k^4 + 0.66*R_v^3*c_k^3 + 0.15*R_v^2*c_k^2 + 0.014*R_v*c_k + 0.00043)*C*DEg)*((-0.66*I*R_v^4*c_k^4 - 0.55*I*R_v^3*c_k^3 - 0.17*I*R_v^2*c_k^2 - 0.021*I*R_v*c_k - 0.00086*I)*C^1.5*R_f + (R_v^4*c_k^4 + 0.66*R_v^3*c_k^3 + 0.15*R_v^2*c_k^2 + 0.014*R_v*c_k + 0.00043)*C*DEg)*r_g/((R_v^6*c_k^6 + 2*R_v^5*c_k^5 + 1.5*R_v^4*c_k^4 + 0.53*R_v^3*c_k^3 + 0.094*R_v^2*c_k^2 + 0.0078*R_v*c_k + 0.00024)*(R_v^2*c_k^2 + 0.50*R_v*c_k + 0.062)^2*C^3.5*DEg*R_f^2)

coeff:


sqrt(gamma_g)